In [1]:
import os
import sys
import pandas as pd
import fnmatch
import json
import re

from pymystem3 import Mystem
m = Mystem()

In [2]:
path = '../'

In [10]:
data = []
for root, dir1, files in os.walk(path):
    for item in fnmatch.filter(files, "*.txt"):
        data.append([root.split('/')[-1], open(os.path.join(root, item)).read()])
data = pd.DataFrame(data)
data.columns = ['tag', 'text']
data.head()

,tag,text
0,fashion,"Я могу легко заявить, что русские с энтузиазмо..."
1,fashion,"Мода - это то, с чем мы имеем дело каждый день..."
2,fashion,"Что такое шопинг? На мой взгляд, это очень сло..."
3,fashion,Сегодня все больше и больше людей начинают зад...
4,fashion,"Мода - это то, с чем мы имеем дело каждый день..."


In [11]:
data.text = data.text.apply(lambda x: re.sub(r'[^\w\s]', ' ', x.lower()))
data.text = data.text.apply(lambda x: re.sub('\d', ' ', x))
data.text = data.text.apply(lambda x: re.sub(r'[\n\r\t]', ' ', x))
data.text = data.text.apply(lambda x: ' '.join(re.split("[^а-я]*", x.lower())))
data.head()

/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


,tag,text
0,fashion,я могу легко заявить что русские с энтузиазмом...
1,fashion,мода это то с чем мы имеем дело каждый день да...
2,fashion,что такое шопинг на мой взгляд это очень сложн...
3,fashion,сегодня все больше и больше людей начинают зад...
4,fashion,мода это то с чем мы имеем дело каждый день да...


In [12]:
data.text = data.text.apply(lambda x: m.lemmatize(x))
data.text = data.text.apply(lambda x: [i for i in x if ' ' not in i])
data.head()

,tag,text
0,fashion,"[я, мочь, легко, заявлять, что, русский, с, эн..."
1,fashion,"[мода, это, то, с, что, мы, иметь, дело, кажды..."
2,fashion,"[что, такой, шопинг, на, мой, взгляд, это, оче..."
3,fashion,"[сегодня, все, много, и, много, человек, начин..."
4,fashion,"[мода, это, то, с, что, мы, иметь, дело, кажды..."


In [13]:
uni = []
for i in data.text:
    uni += i
uni = sorted(list(set(uni)))

In [14]:
num2v = {}
v2num = {}
for i, v in enumerate(uni):
    v2num[v] = i
    num2v[i] = v

In [9]:
json.dump(num2v, open('../data/num2v.json', 'w'))
json.dump(v2num, open('../data/v2num.json', 'w'))

In [246]:
data.text = data.text.apply(lambda x: ' '.join(str(v2num[i]) for i in x))
data.head()

,tag,text
0,fashion,1244 524 443 322 1214 928 932 1236 1182 296 74...
1,fashion,512 1238 1089 932 1214 528 360 224 386 226 212...
2,fashion,1214 1068 1225 531 517 108 1238 688 977 148 10...
3,fashion,949 158 507 340 507 1202 559 304 617 512 528 1...
4,fashion,512 1238 1089 932 1214 528 360 224 386 226 212...


In [247]:
data.to_csv('../data/prepared.csv', index=False)